# Importing Libraries

In [ ]:
import cv2
import time
import numpy as np
from os import listdir
from os.path import isfile, join
import boto3
import os 
import subprocess
import time

In [ ]:
name = input("Enter Your Friend's Name: ")

# 1. Creating DataSet

In [ ]:
# Load HAAR face classifier
Classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [ ]:
#Initialize Internal Webcam 
cap = cv2.VideoCapture(0)

In [ ]:
#Detection of the face 
#(images passed in this function one by one and then it returns the cropped face)

def face_detection(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = Classifier.detectMultiScale(photo)
    
    if len(faces)==0:
        return None
    
    #else part which crop the input image 
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
    return cropped_face

In [ ]:
#Collecting 100 samples of your face from webcam input (Required data)
count = 0
while True:
    #Read single image of the face 
    ret, photo = cap.read()
    
    if face_detection(photo) is not None:
        # face found count increase
        count +=1
        # resizing every image
        face = cv2.resize(face_detection(photo), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        #save the data file in the specified directory
        #image saved as count_number.jpg in particular directory path
        file_path = './faces/user/'+ str(count)+'.jpg'
        cv2.imwrite(file_path, face)
        
        #putting the count text on image to display the live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        pass

    if cv2.waitKey(1) == 13 or count == 100: 
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")
print("+-----------------------------+")
print("Dataset Successfully Created!")

# 2. Train the model

In [ ]:
#Get the data to train from the specified directory
data_path = './faces/user/'

In [ ]:
#list of images 
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

In [ ]:
# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
model  = cv2.face_LBPHFaceRecognizer.create()

# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained successefully!!")

# 3. Adding Functions

In [ ]:
def createInstance():
    os.system('aws ec2 run-instances --image-id ami-0ad704c126371a549 --count 1 --instance-type t2.micro --key-name XXXXXXXX --security-group-ids sg-0029df2a2a568ca7d')
    print("ec2-instance Created!")

def createVolume():
    az = subprocess.getoutput('aws ec2 describe-instances --filters Name=instance-state-name,Values=running --query "Reservations[].Instances[].Placement.AvailabilityZone" --output text')
    os.system('aws ec2 create-volume --availability-zone  {} --size 5'.format(az))
    print("EBS of 5GiB Volume created!")

def attachVolume():
    for i in range(20):
        time.sleep(1)
        print("Attaching...")
    instance_id = subprocess.getoutput('aws ec2 describe-instances --filters Name=instance-state-name,Values=running --query "Reservations[].Instances[].InstanceId" --output text')
    volume_id = subprocess.getoutput('aws ec2 describe-volumes --filters Name=size,Values=5 --query "Volumes[*].VolumeId" --output text')
    os.system('aws ec2 attach-volume --device /dev/xvdc --instance-id {} --volume-id {}'.format(instance_id, volume_id))
    print("Volume attached to the Instance!")

# 4. Run the Model

In [ ]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces == ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

In [ ]:
import os
# Open Internal Webcam
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    image, face = face_detector(frame)    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        # "results" comprises of a tuple containing the label and the confidence value
        results = model.predict(face)  #passing face for prediction
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Correct'
            cv2.putText(image, display_string, (185, 120), cv2.FONT_ITALIC, 1, (255,40,150), 2)
        
        if confidence >= 90:
            cv2.putText(image,"Face Found!",(190, 80) , cv2.FONT_ITALIC,1, (0,0,255), 2)
            cv2.putText(image, name, (175,440), cv2.FONT_ITALIC, 1, (0,255,0), 2)
            cv2.imshow('Detecting Face', image )
            isConfident = True
                        
            
            
        else:
            cv2.putText(image, "User Not Found!", (175,440), cv2.FONT_ITALIC, 1, (0,0,255), 2)
            cv2.imshow('Detecting Face', image )

    except:
        cv2.putText(image, "No Face Found!", (185, 100) , cv2.FONT_ITALIC,1, (0,0,255), 2)
        cv2.putText(image, "Looking for face..", (175, 440), cv2.FONT_ITALIC, 1, (0,0,255), 2)
        cv2.imshow('Detecting Face', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
if isConfident == True:
    print("Face Detected!")
    print("-----------------------")
    createInstance()
    print("#########################")
    createVolume()
    print("#########################")
    attachVolume()
    print("#########################")
    print("GREAT SUCCESS!!")